In [1]:
import os
import json
import logging
import numpy as np

In [2]:
def load_json_from_lib(nome_file, local = False):
    # Usa __file__ per ottenere il percorso della directory del file corrente
    if not local:
        file_path = os.path.join(os.path.dirname(__file__), nome_file)
    else:
        file_path = nome_file
    with open(file_path, 'r') as file:
        return json.load(file)

In [3]:
import sys
import os

# Append the parent directory to sys.path
sys.path.append(os.path.abspath('..'))

# Now you can import the module
from llm_utils import api_keys


INFO:datasets:PyTorch version 2.7.0 available.


Using default model: meta-llama/llama-4-maverick-17b-128e-instruct
/root/projects/nano-graphrag/biomedical/api_keys.json
Ollama base url http://0.0.0.0:11434


In [4]:
print("\n\n<<<<<<<<<<<<< No-RAG Reply >>>>>>>>>>>>>>>")
GROQ_API_KEY = api_keys["groq"]
print("GROQ_API_KEY:", GROQ_API_KEY)
MODEL = "meta-llama/llama-4-scout-17b-16e-instruct"
# MODEL = "llama-3.1-8b-instant"

# Test Native LLM response
from groq import Groq

client = Groq(api_key=GROQ_API_KEY)
question = "What is the capital of France?"

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": question},
    ],
    stream=False
)

print(response.choices[0].message.content)
print("<<< ----------------- >>>")



<<<<<<<<<<<<< No-RAG Reply >>>>>>>>>>>>>>>
GROQ_API_KEY: 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


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


The capital of France is Paris.
<<< ----------------- >>>


# Data Preparation

In [ ]:
!pip install groq

In [49]:
import pandas as pd

dataset_df = pd.read_csv("../datasets/working_dataset_1000.csv")
dataset_df

,pubmed_id,INTRO,METHODS,RESULTS,DISCUSS,text_length,TOPIC_LABEL
0,30419927,Background<SEP>The Mediterranean diet (MeD) is...,Methods<SEP>Study design and subjects<SEP>The ...,Results<SEP>Population characteristics<SEP>The...,Discussion<SEP>Among the different dietetic pa...,21979,"['OBWCCE', 'EATBTS', 'FOODIN']"
1,30032228,Introduction<SEP>Overweight and obesity have b...,Methods<SEP>This systematic review was perform...,Results<SEP>Articles Identified<SEP>The initia...,Discussion<SEP>The purpose of this review was ...,32452,"['OBWCCE', 'EATBTS']"
2,21799482,"INTRODUCTION<SEP>In recent years, the prevalen...",METHODS AND PROCEDURES<SEP>Study population<SE...,RESULTS<SEP>Characteristics of the study parti...,"DISCUSSION<SEP>In our study, we found that var...",22444,"['OBWCCE', 'EATBTS']"
3,24516669,Introduction<SEP>Osteoporosis and obesity are ...,Materials and Methods<SEP>Subjects<SEP>Two pop...,Results<SEP>The general and clinical character...,Discussion<SEP>Obesity is an established risk ...,21427,['OBWCCE']
4,19079261,"Obesity is a major public health problem, resu...",METHODS<SEP>Study design<SEP>This study is com...,RESULTS<SEP>Initial meta-analysis of GWAS stud...,DISCUSSION<SEP>Through meta-analysis of GWA da...,35778,['OBWCCE']
...,...,...,...,...,...,...,...
995,24852944,Introduction<SEP>Adoptive immunotherapy is a p...,Materials and Methods<SEP>T cell culture<SEP>A...,Results and Discussions<SEP>The coupling of pr...,NaN,19075,['FOODAL']
996,28231819,Background<SEP>Targeted cancer immunotherapies...,Methods<SEP>Tumor samples and sequencing<SEP>S...,Results<SEP>Tumor mutanomes are unique<SEP>We ...,Discussion<SEP>Our data reveal that inter-indi...,22078,['FOODAL']
997,31676868,"In type 1 diabetes (T1D), early inflammation o...",Methods:<SEP>Human Islets and EndoC-βH1<SEP>Hu...,Results<SEP>Pro-inflammatory cytokines impact ...,Discussion:<SEP>Our work illustrates the human...,43718,['FOODAL']
998,31506281,Introduction<SEP>Germinal centers (GCs) form i...,Materials and methods<SEP>Animals and treatmen...,Results<SEP>TGFβ signaling occurs in GCBs and ...,"Discussion<SEP>In this study, we showed that T...",42774,['FOODAL']


In [57]:
dataset_df['TOPIC_LABEL'].unique()

default_entity_types = [
    "Gene",
    "Genetic Variant",
    "Transcript",
    "Protein",
    "Nutrient",
    "Food",
    "Dietary Pattern",
    "Physiological Process",
    "Metabolic Pathway",
    "Molecular Interaction",
    "Environmental Factor",
    "Disease",
]


In [50]:
# df_FOODAL = dataset_df[dataset_df["TOPIC_LABEL"] == ["FOODAL"]]
# df_OBWCCE = dataset_df[dataset_df["TOPIC_LABEL"] == ["OBWCCE"]]
# df_EATBTS = dataset_df[dataset_df["TOPIC_LABEL"] == ["EATBTS"]]

df_FOODAL = dataset_df[dataset_df['TOPIC_LABEL'].apply(lambda x: 'FOODAL' in x)]
df_OBWCCE = dataset_df[dataset_df['TOPIC_LABEL'].apply(lambda x: 'OBWCCE' in x)]
df_EATBTS = dataset_df[dataset_df['TOPIC_LABEL'].apply(lambda x: 'EATBTS' in x)]
df_VITMIN = dataset_df[dataset_df['TOPIC_LABEL'].apply(lambda x: 'VITMIN' in x)]
df_FOODIN = dataset_df[dataset_df['TOPIC_LABEL'].apply(lambda x: 'FOODIN' in x)]

In [ ]:
working_dir = "/content/drive/MyDrive/workshop-ECAI/"
working_dir = os.getcwd() + "/documents_from_working_dataset/"
df_FOODAL.sample(5)['RESULTS'].to_json(working_dir+"foodal.json", orient="records")
df_OBWCCE.sample(5)['RESULTS'].to_json(working_dir+"obwcce.json", orient="records")
df_EATBTS.sample(5)['RESULTS'].to_json(working_dir+"eatbts.json", orient="records")
df_VITMIN.sample(5)['RESULTS'].to_json(working_dir+"vitmin.json", orient="records")
df_FOODIN.sample(5)['RESULTS'].to_json(working_dir+"foodin.json", orient="records")

In [51]:
# prompt: read a json file and count the element
import json

working_dir = os.getcwd() + "/documents_from_working_dataset/"
file_path = working_dir+"foodal.json"  # Replace with the actual path to your JSON file

with open(file_path, 'r') as file:
  data = json.load(file)

# Assuming the JSON file contains a list or a dictionary
if isinstance(data, list):
  element_count = len(data)
  print(f"The JSON file contains {element_count} elements in the list.")
elif isinstance(data, dict):
  element_count = len(data)
  print(f"The JSON file contains {element_count} key-value pairs in the dictionary.")
else:
  print("The JSON file does not contain a list or a dictionary at the top level.")

The JSON file contains 5 elements in the list.


In [ ]:
from mychatgpt import bot
men = bot("mendel",model=4)
m='''You have to bias this prompt, toward Biomedical entities and context on this entities relationship. 
This are the entities of our KNowledge Graph
default_entity_types = [
    "Gene",
    "Genetic Variant",
    "Transcript",
    "Protein",
    "Nutrient",
    "Food",
    "Dietary Pattern",
    "Physiological Process",
    "Metabolic Pathway",
    "Molecular Interaction",
    "Environmental Factor",
    "Disease",
]

This is part of a script for question generation that will be used to generate high-level questions for a corpus of genetic  documents realted to nutrition (nutrigentics).

goal_PROMPT = """ Given a corpus description and the parameters K (number of users), N (number of tasks per user), and M (number of high-level questions per (user, task) pair), your task is to generate high-level questions that assess global understanding of the corpus.

"""

'''
men.c(m)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


When biasing the question generation script toward biomedical entities, especially in the context of nutrigenetics, it's crucial to incorporate the intricate interactions between the specified entities such as genes, nutrients, diseases, and metabolic pathways. This targeted approach should reflect a deep understanding of how these entities interact within biological systems and their implications for nutritional genetics. Here's how the goal prompt can be adapted and elaborated to align with these considerations:

---

**Biomedical Goal Prompt:**

In the context of nutrigenetics, where we analyze the interactions between genetic variants and nutritional elements, your task is to generate high-level questions focusing on the complex relationships and effects among the following entities: Gene, Genetic Variant, Transcript, Protein, Nutrient, Food, Dietary Pattern, Physiological Process, Metabolic Pathway, Molecular Interaction, Environmental Factor, and Disease. These questions should a

In [10]:
role_PROMPT = """
      You are a helpful assistant responsible for generating evaluation questions
      from a given corpus using a structured procedure.
      
      In the context of nutrigenetics, where we analyze the interactions between genetic variants and nutritional elements, your task is to generate high-level questions focusing on the complex relationships and effects among the following entities: Gene, Genetic Variant, Transcript, Protein, Nutrient, Food, Dietary Pattern, Physiological Process, Metabolic Pathway, Molecular Interaction, Environmental Factor, and Disease. These questions should aim to deepen the global understanding of how nutritional factors interact with genetic makeup to influence health and disease outcomes.
      """
# goal_PROMPT = """
#       Given a corpus description and the parameters K (number of users), N (number of tasks per user),
#       and M (number of high-level questions per (user, task) pair), your task is to generate high-level questions
#       that assess global understanding of the corpus.
#       """
goal_PROMPT = """
Given a corpus description and the parameters K (number of users), N (number of tasks per user), and M (number of high-level questions per (user, task) pair), your task is to craft questions that illuminate the interconnectedness and impact of these biomedical entities. Consider the following angles to guide your question generation:
entities: Gene, Genetic Variant, Transcript, Protein, Nutrient, Food, Dietary Pattern, Physiological Process, Metabolic Pathway, Molecular Interaction, Environmental Factor, and Disease. These questions should aim to deepen the global understanding of how nutritional factors interact with genetic makeup to influence health and disease outcomes.

Your questions should explore the following themes:
1. **Genomic Influence on Nutrition:** How do specific genetic variants influence individual responses to nutrients and dietary patterns? Explore effects on metabolic pathways and physiological processes.

2. **Nutritional Impact on Gene Expression:** In what ways do particular nutrients or dietary patterns affect gene expression, transcript production, and protein synthesis? Assess the downstream impact on health and disease.

3. **Disease and Genetic-Nutrient Interactions:** How do dietary factors interact with genes to modulate disease risk or progression? Consider molecular interactions and potential environmental influences.

4. **Metabolic Pathway Insights:** What roles do metabolic pathways and molecular interactions play in mediating the effects of nutrition-genetic interactions on physiological processes and disease outcomes?

5. **Environmental and Nutrient Modulation:** How do environmental factors alter the interactions between genes and nutrients, potentially influencing health outcomes and disease susceptibility? 

Your questions should endeavor to connect these entities within a comprehensive framework that reflects the complexity of nutrigenomic research, fostering a robust understanding of the underlying biological mechanisms.ticError

"""
# generation_step_PROMPT = """
#       Your generation should follow these steps:
#       1. Create K user personas based on the corpus, including their role/background and motivation for using the corpus.
#       2. For each user, define N realistic and relevant tasks they would perform using the corpus.
#       3. For each (user, task) pair, generate M high-level questions that:
#           - Require comprehensive understanding of the whole corpus.
#           - Do NOT depend on low-level facts or specific data points (e.g., names, dates, figures)."""

generation_step_PROMPT = """
      Your generation should follow these steps:
      1. Create 3 user personas based on the corpus, including their role/background and motivation for using the corpus.
            user 1: a researcher in nutrigenetics, interested in understanding how genetic variants influence dietary responses.
            user 2: a healthcare professional looking to apply nutrigenetic insights in clinical practice.
            user 3: a customer interested in personalized nutrition based on genetic information.
      2. For each user, define N realistic and relevant tasks they would perform using the corpus.
      3. For each (user, task) pair, generate M high-level questions that:
          - Require comprehensive understanding of the whole corpus.
          - Do NOT depend on low-level facts or specific data points (e.g., names, dates, figures)."""
output_PROMPT = """
      User 1: [Persona description]
      Task 1:
      Q1. ...
      Q2. ...
      ...
      Task 2:
      Q1. ...
      ...
      User 2: ...
      ...
      User K: ..."""

output_format_PROMPT = """
      {{
        "corpus_description": "<your summary of the corpus>",
        "generated_questions": "<your structured output as described above>"
      }}"""

In [12]:
def generate_questions(corpus_description, K, N, M):
  PROMPT = f"""
  ---Role--
  {role_PROMPT}

  ---Goal--
  {goal_PROMPT}

  {generation_step_PROMPT}

  Your output should be structured as follows:
  {output_PROMPT}

  Format your response as a JSON object with the following structure:
  {output_format_PROMPT}

  ---Corpus Description--
  {corpus_description}
  ---Parameters--
  Number of Users (K): {K}
  Tasks per User (N): {N}
  Questions per (User, Task) (M): {M}

  Output:
  """
  return PROMPT

In [101]:
import os
from groq import Groq

MODEL = "meta-llama/llama-4-scout-17b-16e-instruct"
GROQ_API_KEY = "***"
GROQ_API_KEY = api_keys["groq"]

client = Groq(
    api_key=GROQ_API_KEY,
)

In [ ]:
with open('/content/drive/MyDrive/workshop-ECAI/groq_api.txt', 'w') as f:
  f.write(GROQ_API_KEY)

In [ ]:
working_dir = "/content/drive/MyDrive/workshop-ECAI/"
working_dir = os.getcwd() + "/documents_from_working_dataset/"
foodal_docs = json.load(open(working_dir+"foodal.json"))
obwcce_docs = json.load(open(working_dir+"obwcce.json"))
eatbts_docs = json.load(open(working_dir+"eatbts.json"))
vitmin_docs = json.load(open(working_dir+"vitmin.json"))
foodin_docs = json.load(open(working_dir+"foodin.json"))

In [68]:
docs_list = [foodal_docs, obwcce_docs, eatbts_docs, vitmin_docs, foodin_docs]
labels = ["foodal", "obwcce", "eatbts", "vitmin", "foodin"]

for i, doc in enumerate(docs_list):
    corpus_description_PROMPT = f"""
    ---Role--
    You are a helpful assistant tasked with generating a description of a corpus based on the results of 5 randomly selected academic papers.

    ---Goal--
    Given a set of 5 academic papers, your task is to summarize the key findings from each paper and combine this information into a coherent corpus description. The description should highlight common themes, methodologies used, and major conclusions drawn across the papers.

    ---Context--
    In the context of nutrigenetics, where we analyze the interactions between genetic variants and nutritional elements, your task is to generate high-level questions focusing on the complex relationships and effects among the following entities: Gene, Genetic Variant, Transcript, Protein, Nutrient, Food, Dietary Pattern, Physiological Process, Metabolic Pathway, Molecular Interaction, Environmental Factor, and Disease. These questions should aim to deepen the global understanding of how nutritional factors interact with genetic makeup to influence health and disease outcomes.

    ---Papers---
    1. {doc[0]}
    2. {doc[1]}
    3. {doc[2]}
    4. {doc[3]}
    5. {doc[4]}

    ---Instructions---
    For each paper:
    - Briefly summarize the study's objective.
    - Describe the methodology used.
    - List the main findings or results highlighting biomedical entities.

    After analyzing all 5 papers:
    - Identify common themes across the papers.
    - Highlight any significant differences in methodologies or results.
    - Provide an overall description of the corpus that reflects the combined knowledge and insights from the papers.

    ---Output Format---
    Return your response as a single text object with the following structure:

    <Detailed description of the corpus based on the 5 papers>

    ---Output---
    **Corpus Description**

    """


    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": corpus_description_PROMPT
            }
        ],
        model=MODEL,
    )

    answer = chat_completion.choices[0].message.content

    # print(answer)


    # prompt: write the foodal_answer in a file text in my folder in drive

    corpus_description = answer.split("**")[-1]
    print(corpus_description)

    with open(working_dir+f'{labels[i]}_answer.txt', 'w') as f:
       f.write(corpus_description)

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"




The corpus provides a comprehensive overview of the complex relationships between genetic variants, nutritional elements, and disease outcomes. The papers highlight the importance of considering genetic variants in understanding responses to environmental factors and disease susceptibility. The studies demonstrate the use of various methodologies, including genome-wide association studies, genetic analysis, and multifactor dimensionality reduction analysis, to investigate gene-environment interactions. The corpus contributes to our understanding of the mechanisms underlying nutrigenetics and its applications in personalized medicine.


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"




The corpus provides a comprehensive overview of the current state of knowledge in nutrigenetics, highlighting the complex interactions between genetic variants, nutritional elements, and their effects on health and disease outcomes. The combined knowledge from these papers underscores the importance of personalized nutrition, inflammation, and disease susceptibility, and highlights the need for further research in this field.


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"




The corpus provides a comprehensive understanding of the complex interactions between genetic variants, nutritional elements, and disease outcomes. The papers collectively highlight the significance of genetics, nutrition, and pharmacogenetics in understanding disease susceptibility and response to treatment. The corpus covers various biomedical entities, including genes, nutrients, and diseases, and demonstrates the importance of considering these factors in disease prevention and management. Overall, the corpus offers valuable insights into the intricate relationships between genes, nutrients, and health, ultimately contributing to the advancement of personalized medicine and nutrigenetics.


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"




The corpus provides a comprehensive understanding of the complex relationships between genetic variants, nutritional elements, and their impact on health and disease outcomes. The papers highlight the importance of gene-nutrient interactions in modulating physiological processes, metabolic pathways, and disease susceptibility. The findings have implications for personalized nutrition and disease prevention strategies. The corpus also underscores the need for further research into the mechanisms underlying gene-nutrient interactions and their effects on human health.


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"




The corpus provides insights into the complex relationships between genetic variants, nutritional elements, and human health and disease outcomes. The papers highlight the importance of understanding the genetic basis of disease susceptibility and progression, as well as the role of environmental factors, such as nutrition and lifestyle interventions. The corpus covers a range of biomedical entities, including genes, genetic variants, proteins, and diseases, and demonstrates the utility of functional studies in elucidating the effects of genetic variants on protein activity and cellular processes. Overall, the corpus contributes to our understanding of the intricate relationships between genetic and environmental factors and their impact on human health and disease outcomes.


# Question Generation

In [70]:
K = 3
N = 5
M = 5
# working_dir = "/content/drive/MyDrive/workshop-ECAI/"
working_dir = os.getcwd() + "/documents_from_working_dataset/"

docs_list = [foodal_docs, obwcce_docs, eatbts_docs, vitmin_docs, foodin_docs]
labels = ["foodal", "obwcce", "eatbts", "vitmin", "foodin"]

for i, doc in enumerate(docs_list):
  with open(working_dir+f'{labels[i]}_answer.txt', 'r') as file:
    corpus_description = file.read()

  prompt = generate_questions(corpus_description, K, N, M)

  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt
          }
      ],
      model=MODEL,
  )

  answer = chat_completion.choices[0].message.content

  print(answer)

  question_dir = os.getcwd() + "/questions_files/"
  with open(question_dir+f'{labels[i]}_generated_question.txt', 'w') as f:
    f.write(answer)

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Here is the output:

```
{
  "corpus_description": "The corpus provides a comprehensive overview of the complex relationships between genetic variants, nutritional elements, and disease outcomes. The papers highlight the importance of considering genetic variants in understanding responses to environmental factors and disease susceptibility. The studies demonstrate the use of various methodologies, including genome-wide association studies, genetic analysis, and multifactor dimensionality reduction analysis, to investigate gene-environment interactions. The corpus contributes to our understanding of the mechanisms underlying nutrigenetics and its applications in personalized medicine.",
  "generated_questions": "
  User1: A researcher in nutrigenetics, interested in understanding how genetic variants influence dietary responses.
  Task1: Investigating the impact of genetic variants on nutrient metabolism
  Q1. How do specific genetic variants in the genes involved in carbohydrate metab

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Here is the output:

```
{
  "corpus_description": "The corpus provides a comprehensive overview of the current state of knowledge in nutrigenetics, highlighting the complex interactions between genetic variants, nutritional elements, and their effects on health and disease outcomes. The combined knowledge from these papers underscores the importance of personalized nutrition, inflammation, and disease susceptibility, and highlights the need for further research in this field.",
  "generated_questions": "
  User1: A researcher in nutrigenetics, interested in understanding how genetic variants influence dietary responses.

  Task1: Investigate the relationship between genetic variants and nutrient responses
  Q1. How do specific genetic variants affect the metabolism of nutrients and influence physiological processes?
  Q2. What are the downstream effects of genetic variants on metabolic pathways and disease susceptibility?
  Q3. How do genetic variants interact with dietary patterns to

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Here is the output:

```
{
  "corpus_description": "The corpus provides a comprehensive understanding of the complex interactions between genetic variants, nutritional elements, and disease outcomes. The papers collectively highlight the significance of genetics, nutrition, and pharmacogenetics in understanding disease susceptibility and response to treatment. The corpus covers various biomedical entities, including genes, nutrients, and diseases, and demonstrates the importance of considering these factors in disease prevention and management. Overall, the corpus offers valuable insights into the intricate relationships between genes, nutrients, and health, ultimately contributing to the advancement of personalized medicine and nutrigenetics.",
  "generated_questions": "
  
  User1: A researcher in nutrigenetics, interested in understanding how genetic variants influence dietary responses.
  Task1: Investigating the relationship between genetic variants and nutrient response
  Q1. How

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Here is the output:

```
{
  "corpus_description": "The corpus provides a comprehensive understanding of the complex relationships between genetic variants, nutritional elements, and their impact on health and disease outcomes. The papers highlight the importance of gene-nutrient interactions in modulating physiological processes, metabolic pathways, and disease susceptibility. The findings have implications for personalized nutrition and disease prevention strategies. The corpus also underscores the need for further research into the mechanisms underlying gene-nutrient interactions and their effects on human health.",
  "generated_questions": "
  User1: A researcher in nutrigenetics, interested in understanding how genetic variants influence dietary responses.
  Task1: Investigating the relationship between genetic variants and nutrient response.
  Q1. How do specific genetic variants affect the metabolism of nutrients and their subsequent impact on physiological processes?
  Q2. What

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Here is the output:

```
{
  "corpus_description": "The corpus provides insights into the complex relationships between genetic variants, nutritional elements, and human health and disease outcomes. The papers highlight the importance of understanding the genetic basis of disease susceptibility and progression, as well as the role of environmental factors, such as nutrition and lifestyle interventions. The corpus covers a range of biomedical entities, including genes, genetic variants, proteins, and diseases, and demonstrates the utility of functional studies in elucidating the effects of genetic variants on protein activity and cellular processes. Overall, the corpus contributes to our understanding of the intricate relationships between genetic and environmental factors and their impact on human health and disease outcomes.",
  "generated_questions": "
  User1: A researcher in nutrigenetics, interested in understanding how genetic variants influence dietary responses.
  Task1: Invest

# Restructuring questions in a DataFrame


In [71]:
question_dir = os.getcwd() + "/questions_files/"

with open(question_dir+'foodal_generated_question.txt', 'r') as file:
  foodal_questions = file.read()

with open(question_dir+'obwcce_generated_question.txt', 'r') as file:
  obwcce_questions = file.read()

with open(question_dir+'eatbts_generated_question.txt', 'r') as file:
  eatbts_questions = file.read()

with open(question_dir+'vitmin_generated_question.txt', 'r') as file:
  vitmin_questions = file.read()

with open(question_dir+'foodin_generated_question.txt', 'r') as file:
  foodin_questions = file.read()

In [ ]:
print(foodal_questions)

In [72]:
import pandas as pd
import re

# Step 1: Read file as plain text
#-- Fatto nella cella sopra per tutti

questions = [foodin_questions, foodal_questions, obwcce_questions, eatbts_questions, vitmin_questions]
categories = ["FOODIN", "FOODAL", "OBWCCE", "EATBTS", "VITMIN"]

data = {}

for i, q in enumerate(questions):

    # Step 2: Manually extract the 'generated_questions' content
    match = re.search(r'"generated_questions"\s*:\s*"(?P<content>.*)"\s*}', q, re.DOTALL)
    if not match:
        raise ValueError("Could not find 'generated_questions' content in the file.")

    content = match.group("content")

    # Step 3: Unescape escaped quotes and normalize the content
    content = content.replace('\\"', '"')

    # Step 4: Parse lines
    lines = content.splitlines()


    category = categories[i]  # or derive from file name
    print(f"Processing category: {category}")

    rows = []
    current_user = None
    current_task = None
    questions = []

    for line in lines:
        line = line.strip()
        if not line:
            continue
        if line.startswith("User"):
            current_user = line.split(":", 1)[1].strip()
        elif line.startswith("Task"):
            if current_task and questions:
                rows.append({
                    "Category": category,
                    "USER Description": current_user,
                    "TASK Description": current_task,
                    "Questions": questions
                })
                questions = []
            current_task = line.split(":", 1)[1].strip()
        elif re.match(r"Q\d+\.", line):
            questions.append(line)

    # Append the last block
    if current_task and questions:
        rows.append({
            "Category": category,
            "USER Description": current_user,
            "TASK Description": current_task,
            "Questions": questions
        })
    
    data[category] = rows



Processing category: FOODIN
Processing category: FOODAL
Processing category: OBWCCE
Processing category: EATBTS
Processing category: VITMIN


In [73]:
# Step 5: Create DataFrame
df_foodal = pd.DataFrame(data["FOODAL"])
df_obwcce = pd.DataFrame(data["OBWCCE"])
df_eatbts = pd.DataFrame(data["EATBTS"])
df_vitmin = pd.DataFrame(data["VITMIN"])
df_foodin = pd.DataFrame(data["FOODIN"])
df_foodal

,Category,USER Description,TASK Description,Questions
0,FOODAL,"A researcher in nutrigenetics, interested in u...",Investigating the impact of genetic variants o...,[Q1. How do specific genetic variants in the g...
1,FOODAL,"A researcher in nutrigenetics, interested in u...",Exploring the relationship between genetic var...,[Q1. What is the relationship between genetic ...
2,FOODAL,"A researcher in nutrigenetics, interested in u...",Analyzing the impact of genetic variants on di...,[Q1. How do genetic variants in the genes invo...
3,FOODAL,"A researcher in nutrigenetics, interested in u...",Investigating the role of epigenetics in nutri...,[Q1. How do epigenetic modifications affect th...
4,FOODAL,A healthcare professional looking to apply nut...,Developing personalized nutrition recommendations,[Q1. How can genetic variants be used to tailo...
5,FOODAL,A healthcare professional looking to apply nut...,Integrating nutrigenetics into clinical practice,[Q1. What are the clinical implications of gen...
6,FOODAL,A healthcare professional looking to apply nut...,Understanding the role of nutrigenetics in dis...,[Q1. How do genetic variants in the genes invo...
7,FOODAL,A healthcare professional looking to apply nut...,Exploring the relationship between genetic var...,[Q1. How do genetic variants in the genes invo...
8,FOODAL,A healthcare professional looking to apply nut...,Investigating the impact of epigenetics on cli...,[Q1. How do epigenetic modifications affect th...
9,FOODAL,A customer interested in personalized nutritio...,Developing evidence-based guidelines for nutri...,[Q1. What are the current evidence-based guide...


In [74]:
# prompt: append all these dataframe together: df_foodal

df_combined = pd.concat([df_foodal, df_obwcce, df_eatbts, df_vitmin, df_foodin], ignore_index=True)

In [84]:

df_combined.to_pickle("questions_dataframe_v3.pkl")
df_combined

,Category,USER Description,TASK Description,Questions
0,FOODAL,"A researcher in nutrigenetics, interested in u...",Investigating the impact of genetic variants o...,[Q1. How do specific genetic variants in the g...
1,FOODAL,"A researcher in nutrigenetics, interested in u...",Exploring the relationship between genetic var...,[Q1. What is the relationship between genetic ...
2,FOODAL,"A researcher in nutrigenetics, interested in u...",Analyzing the impact of genetic variants on di...,[Q1. How do genetic variants in the genes invo...
3,FOODAL,"A researcher in nutrigenetics, interested in u...",Investigating the role of epigenetics in nutri...,[Q1. How do epigenetic modifications affect th...
4,FOODAL,A healthcare professional looking to apply nut...,Developing personalized nutrition recommendations,[Q1. How can genetic variants be used to tailo...
...,...,...,...,...
70,FOODIN,A customer interested in personalized nutritio...,Understanding the basics of nutrigenomics,[Q1. What are the key concepts in nutrigenomic...
71,FOODIN,A customer interested in personalized nutritio...,Exploring the relationship between genetic var...,[Q1. How do genetic variants affect the metabo...
72,FOODIN,A customer interested in personalized nutritio...,Investigating the role of epigenetics in nutri...,"[Q1. How do environmental factors, such as nut..."
73,FOODIN,A customer interested in personalized nutritio...,Analyzing the impact of nutrient-gene interact...,"[Q1. How do specific nutrients, such as folate..."


In [85]:
for desc in df_combined['USER Description'].drop_duplicates():
    print(desc,"\n------")

A researcher in nutrigenetics, interested in understanding how genetic variants influence dietary responses. 
------
A healthcare professional looking to apply nutrigenetic insights in clinical practice. 
------
A customer interested in personalized nutrition based on genetic information. 
------


In [88]:
for desc in df_combined['Questions'].drop_duplicates():
    print(desc,"\n------")

['Q1. How do specific genetic variants in the genes involved in carbohydrate metabolism affect the response to dietary carbohydrates?', 'Q2. What are the effects of genetic variants on the expression of enzymes involved in fatty acid metabolism?', 'Q3. How do genetic variants in the genes encoding for nutrient transporters influence nutrient uptake and utilization?', 'Q4. Can genetic variants in the genes involved in epigenetic regulation modulate the response to dietary interventions?', 'Q5. How do genetic variants in the genes involved in inflammation affect the response to dietary patterns and disease risk?'] 
------
['Q1. What is the relationship between genetic variants in the genes involved in taste perception and dietary preferences?', 'Q2. How do genetic variants in the genes involved in satiety and appetite regulation influence food choices?', 'Q3. Can genetic variants in the genes involved in nutrient sensing affect the response to dietary restriction?', 'Q4. How do genetic v

In [86]:
for desc in df_combined['TASK Description'].drop_duplicates():
    print(desc,"\n------")

Investigating the impact of genetic variants on nutrient metabolism 
------
Exploring the relationship between genetic variants and dietary patterns 
------
Analyzing the impact of genetic variants on disease risk 
------
Investigating the role of epigenetics in nutrigenetics 
------
Developing personalized nutrition recommendations 
------
Integrating nutrigenetics into clinical practice 
------
Understanding the role of nutrigenetics in disease prevention 
------
Exploring the relationship between genetic variants and nutritional deficiencies 
------
Investigating the impact of epigenetics on clinical practice 
------
Developing evidence-based guidelines for nutrigenetic testing 
------
Understanding the basics of nutrigenetics 
------
Exploring the role of genetic variants in dietary responses 
------
Investigating the impact of epigenetics on nutrition 
------
Understanding the relationship between genetic variants and disease risk 
------
Applying nutrigenetic insights to daily li

# Use community report summary

In [7]:

import json
cache_dict ={1: 'cache_llama3.1_8b_all-mpnet-base-v2',
             2: 'cache_gemma2_all-mpnet-base-v2',
             3: 'cache_qwen2.5_14b_all-mpnet-base-v2',
             4: 'cache_qwen2_7b_dmis-lab_biobert-v1.1',
             5: 'cache_qwen2_7b_all-mpnet-base-v2',
             6: 'cache_qwen2.5_14b_dmis-lab_biobert-v1.1',
             7: 'cache_llama3.1_8b_dmis-lab_biobert-v1.1',
             8: 'cache_gemma2_dmis-lab_biobert-v1.1'}
n = 2
# community reports
root = os.getcwd()
root = "/root/projects/nano-graphrag/biomedical/ablation_study"
file = os.path.join(root, cache_dict[n], 'kv_store_community_reports.json')
print(f"Loading graph from file: {file}\n")

with open(file, 'r') as f:
    community_reports = json.load(f)

# Funzione per estrarre tutti i report con rating > 8 dal file json caricato
def extract_high_score_reports(community_reports, threshold=8):
    # Ritorna una lista dei valori report_json con rating maggiore della soglia
    return [
        report_data["report_json"]
        for report_data in community_reports.values()
        if float(report_data["report_json"]["rating"]) > threshold
    ]


reports = extract_high_score_reports(community_reports, threshold=7)

high_score_reports_summary = [report["summary"] for report in reports]
len(high_score_reports_summary)

# Make 4 list of the from high_score_reports_summary
batch_size = 4
batches = [high_score_reports_summary[i:i + batch_size] for i in range(0, len(high_score_reports_summary), batch_size)]
# for i, batch in enumerate(batches):
#     with open(f"batch_{i+1}_high_score_reports.json", 'w') as f:
#         json.dump(batch, f, indent=4)

batches[0]
for i, batch in enumerate(batches):
    text = "\n\n".join(batch)

text = """**Corpus Description**\n\n"""+text
print(text)

Loading graph from file: /root/projects/nano-graphrag/biomedical/ablation_study/cache_gemma2_all-mpnet-base-v2/kv_store_community_reports.json

**Corpus Description**

This community explores the relationships between genetic markers, immune system responses, and environmental triggers in Celiac Disease (CD). Key entities include genes like PTPN2 and HLA-DRB1*07:01, which are associated with increased CD risk. Immune cells like T-cells play a crucial role in the inflammatory response observed in CD.  The community highlights the complex interplay between genetic predisposition and environmental factors in the development of CD.

This community explores the relationship between genetic variations, environmental factors, and asthma risk. Key entities include genes (like IL-1A -889C>T and TNF-A 238G>A),  SNPs (RS1898671, RS2227306, RS4073), and lifestyle factors like smoking status. The community reveals associations between certain genetic variations and increased susceptibility to asthm

In [ ]:
K = 3
N = 5
M = 5
# working_dir = "/content/drive/MyDrive/workshop-ECAI/"
working_dir = os.getcwd() + "/documents_from_working_dataset/"

answers =[]
for i, batch in enumerate(batches):
  for i, chunk in enumerate(batch):
    text = "\n\n".join(chunk)

  corpus_description = """**Corpus Description**\n\n"""+text

  corpus_description = batch

  prompt = generate_questions(corpus_description, K, N, M)

  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt
          }
      ],
      model=MODEL,
  )

  answer = chat_completion.choices[0].message.content

  answers.append(answer)
  print(answer)
  print("<<< ----------------- >>>")

  # question_dir = os.getcwd() + "/questions_files/"
  # with open(question_dir+f'{labels[i]}_generated_question.txt', 'w') as f:
  #   f.write(answer)

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Here is the output in the requested JSON format:

```json
{
  "corpus_description": "The corpus explores the complex interactions between genetic variants, nutritional elements, and their impact on health and disease outcomes, focusing on entities such as genes, genetic variants, transcripts, proteins, nutrients, food, dietary patterns, physiological processes, metabolic pathways, molecular interactions, environmental factors, and diseases.",
  "generated_questions": "
  User1: A researcher in nutrigenetics, interested in understanding how genetic variants influence dietary responses.
  Task1: Investigating the impact of genetic variants on nutrient metabolism
  Q1. How do genetic variants in metabolic pathway genes influence the response to dietary interventions aimed at weight management?
  Q2. What is the role of transcriptomics in understanding the effects of nutrient-gene interactions on metabolic health?
  Q3. How do dietary patterns modulate the expression of genes involved in i

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


```json
{
  "corpus_description": "The corpus explores the interplay between genetic factors, nutritional elements, and disease risks, particularly focusing on obesity and type 2 diabetes. It includes key genes such as CDKAL1, DNAJC27, SEC16B, TFAP2B, TCF7L2, WFS1, and FTO, and their interactions with dietary factors, metabolic pathways, and environmental influences.",
  "generated_questions": "
  User1: A researcher in nutrigenetics, interested in understanding how genetic variants influence dietary responses.
  Task1: Investigating genetic influences on nutrient interactions
  Q1. How do genetic variants in the CDKAL1 gene influence an individual's response to high-fiber diets in terms of obesity risk?
  Q2. What is the role of the FTO gene in modulating body weight in response to different dietary patterns?
  Q3. How do variations in the TCF7L2 gene affect glucose metabolism in the context of varying carbohydrate intake?
  Q4. Can specific genetic variants in the SEC16B gene predict

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


...

```json
{
  "corpus_description": "The corpus explores the intricate relationships between genetic variants, nutritional elements, and disease outcomes, with a focus on Age-Related Macular Degeneration (AMD), type 2 diabetes (T2D), and diabetes phenotypes. It highlights the influence of specific genes (e.g., ARMS2, CFH, TCF7L2, PPARΓ, CHI3LI, ENPP1, NOTCH2, GCK, TUFM, SULT1A1) and dietary factors (e.g., fiber and fat intake) on disease susceptibility and progression.",
  "generated_questions": "
  User1: A researcher in nutrigenetics, interested in understanding how genetic variants influence dietary responses.
  Task1: Investigate the role of genetic variants in metabolic pathways related to nutrient response.
  Q1. How do genetic variants in the TCF7L2 gene influence an individual's response to dietary fiber in relation to T2D risk?
  Q2. What is the impact of ARMS2 genetic variants on the metabolic pathway of fatty acid oxidation in the context of AMD?
  Q3. How do variations in the

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Here is the output:

```
{
  "corpus_description": "The corpus explores the relationships between genes, proteins, and metabolic processes influencing various diseases and conditions, including kidney function, glucose regulation, immune system components, inflammatory mediators, and lipid metabolism.",
  "generated_questions": "
  
  User1: A researcher in nutrigenetics, interested in understanding how genetic variants influence dietary responses.
  Task1: Investigating the impact of genetic variants on metabolic pathways
  Q1. How do specific genetic variants in genes like GCKR and ALMS1 influence individual responses to dietary patterns, such as carbohydrate intake, and affect kidney function?
  Q2. What are the effects of genetic variants on the regulation of metabolic pathways, such as those involved in glucose regulation and lipid metabolism?
  Q3. How do genetic variants in immune system components, such as cytokines and transcription factors, impact the response to dietary fact

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


```json
{
  "corpus_description": "The corpus explores the complex interactions between genetic variants, nutritional elements, and environmental factors in influencing health and disease outcomes, with a focus on autoimmune diseases, asthma, and cancer.",
  "generated_questions": "
  
  User1: A researcher in nutrigenetics, interested in understanding how genetic variants influence dietary responses.
  Task1: Investigating genetic influences on nutritional responses
  Q1. How do specific genetic variants, such as those in the PTPN2 gene, affect an individual's response to gluten in the context of Celiac Disease?
  Q2. What are the implications of genetic variations in metabolic pathways on the efficacy of dietary interventions for asthma?
  Q3. How do genetic variants influence the expression of proteins involved in nutrient uptake and metabolism?
  Q4. Can genetic testing identify individuals with a higher risk of adverse reactions to certain dietary components?
  Q5. How do genetic 

In [27]:
import re
import re
text = """ """

# Extract User and Task information and question block (as a list ) from the input
def extract_user_tasks(input_text):
    user_tasks = []
    lines = input_text.strip().split('\n')
    current_user = None
    current_task = None
    questions = []

    for line in lines:
        line = line.strip()
        if line.startswith('User'):
            if current_user and current_task and questions:
                user_tasks.append((current_user, current_task, questions))
            # Esxtract the User code (User1, User2, etc.)
            current_user = line.split(':')[0].strip()
            current_task = None
            questions = []
        elif line.startswith('Task'):
            if current_task and questions:
                user_tasks.append((current_user, current_task, questions))
            current_task = line.split(':')[1].strip()
            questions = []
        elif line.startswith('Q'):
            questions.append(line)

    if current_user and current_task and questions:
        user_tasks.append((current_user, current_task, questions))

    return user_tasks
extract_user_tasks(text)
import pandas as pd
df = pd.DataFrame(extract_user_tasks(text), columns=['User', 'Task', 'Questions'])
df.to_pickle(f'questions_cache_gemma2_all-mpnet-base-v2.pkl')


In [25]:
df

,User,Task,Questions
